# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('data/AviationData.csv',low_memory=False)
print (df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [28]:
from datetime import datetime, timedelta
import pandas as pd

# Load the data
df = pd.read_csv('data/AviationData.csv', low_memory=False)

# Calculate the date 40 years ago from the current date
forty_years_ago = datetime.now() - timedelta(days=40*365)

# Create a filtered copy of the dataframe
df_filtered = df.copy()

# Convert Event.Date column to datetime format
df_filtered['Event.Date'] = pd.to_datetime(df_filtered['Event.Date'])

# Filter only category Airplane, Amateur Built is No, and Event Date not more than 40 years ago
df_filtered = df_filtered[(df_filtered['Aircraft.Category'] == 'Airplane') & 
                          (df_filtered['Amateur.Built'] == 'No') & 
                          (df_filtered['Event.Date'] >= forty_years_ago)]

print(df_filtered.info())

<class 'pandas.core.frame.DataFrame'>
Index: 21428 entries, 11898 to 88886
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                21428 non-null  object        
 1   Investigation.Type      21428 non-null  object        
 2   Accident.Number         21428 non-null  object        
 3   Event.Date              21428 non-null  datetime64[ns]
 4   Location                21422 non-null  object        
 5   Country                 21427 non-null  object        
 6   Latitude                19169 non-null  object        
 7   Longitude               19163 non-null  object        
 8   Airport.Code            13971 non-null  object        
 9   Airport.Name            14058 non-null  object        
 10  Injury.Severity         20615 non-null  object        
 11  Aircraft.damage         20203 non-null  object        
 12  Aircraft.Category       21428 non-null  object 

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [36]:
#Assumption is the total number of passengers is the sum of all injury types plus uninjured
#Fatal injuries/total number of passengers = percent fatalities 
#Serious injuries/total number of passengers = percent serious

import pandas as pd

# Load the data
df = pd.read_csv('data/AviationData.csv', low_memory=False)

# Calculate the estimated number of total passengers
df['total_passengers'] = (df['Total.Fatal.Injuries'] + 
                          df['Total.Serious.Injuries'] + 
                          df['Total.Minor.Injuries'] + 
                          df['Total.Uninjured'])

# Avoid division by zero by filtering out rows with zero total passengers
df = df[df['total_passengers'] > 0]

# Create columns for percent fatal and serious injuries
df['percent_fatal'] = df['Total.Fatal.Injuries'] / df['total_passengers']
df['percent_serious'] = df['Total.Serious.Injuries'] / df['total_passengers']

# Validating the data calculations for myself
# Display the top 10 records sorted by percent fatal injuries
# print("Top 10 records by percent fatal injuries:")
# print(df.sort_values(by='percent_fatal', ascending=False).head(10)[['Event.Id', 'percent_fatal', 'Total.Fatal.Injuries', 'total_passengers']])
# Display the top 10 records sorted by percent serious injuries
# print("\nTop 10 records by percent serious injuries:")
# print(df.sort_values(by='percent_serious', ascending=False).head(10)[['Event.Id', 'percent_serious', 'Total.Serious.Injuries', 'total_passengers']])

print (df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 73276 entries, 0 to 88888
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                73276 non-null  object 
 1   Investigation.Type      73276 non-null  object 
 2   Accident.Number         73276 non-null  object 
 3   Event.Date              73276 non-null  object 
 4   Location                73249 non-null  object 
 5   Country                 73064 non-null  object 
 6   Latitude                22357 non-null  object 
 7   Longitude               22358 non-null  object 
 8   Airport.Code            41348 non-null  object 
 9   Airport.Name            43713 non-null  object 
 10  Injury.Severity         73276 non-null  object 
 11  Aircraft.damage         71138 non-null  object 
 12  Aircraft.Category       27283 non-null  object 
 13  Registration.Number     73041 non-null  object 
 14  Make                    73245 non-null  obj

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [38]:
import pandas as pd

# Load the data
df = pd.read_csv('data/AviationData.csv', low_memory=False)

# Create column for tracking if the aircraft was destroyed or not
df['destroyed'] = df['Aircraft.damage'] == 'Destroyed'

# Count how many aircraft were destroyed
destroyed_count = df['destroyed'].sum()

# Validate the number for myself
print(f"Number of destroyed aircraft entries: {destroyed_count}")

print(df.info())

Number of destroyed aircraft entries: 18623
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [42]:
import pandas as pd

# Load the data
df = pd.read_csv('data/AviationData.csv', low_memory=False)

# Clean up the 'Make' column by capitalizing the first letter of each word
df['Make'] = df['Make'].str.title()

# Get counts of each unique make
make_counts = df['Make'].value_counts()

# Filter makes with 50 or more occurrences
makes_over_50 = make_counts[make_counts >= 50]

# Display the number of unique makes
print("Number of unique makes with 50 or more occurrences:", makes_over_50.shape[0])

# Display each make and its count
print("\nCount of each unique make:")
print(makes_over_50)

Number of unique makes with 50 or more occurrences: 98

Count of each unique make:
Make
Cessna                         27149
Piper                          14870
Beech                           5372
Boeing                          2745
Bell                            2722
                               ...  
Lancair                           52
Boeing Stearman                   51
Smith, Ted Aerostar               51
Flight Design Gmbh                50
Grumman American Avn. Corp.       50
Name: count, Length: 98, dtype: int64


### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [43]:
import pandas as pd

# Load the data
df = pd.read_csv('data/AviationData.csv', low_memory=False)

# Clean up the 'Make' column by capitalizing the first letter of each word
df['Make'] = df['Make'].str.title()

# Get counts of each unique make
make_counts = df['Make'].value_counts()

# Filter makes with 50 or more occurrences
makes_over_50 = make_counts[make_counts >= 50]

# Remove rows with NaN values in the 'Model' column
df_clean = df.dropna(subset=['Model']).copy()

# Determine if each Model is unique to exactly one Make
# Count how many unique Makes are associated with each Model
model_make_counts = df_clean.groupby('Model')['Make'].nunique()

# Create a boolean column indicating if the model is unique to one Make
df_clean['Model_Unique_To_Make'] = df_clean['Model'].map(model_make_counts) == 1

# Create a derived column with a unique identifier combining Make and Model
df_clean['Make_Model_ID'] = df_clean['Make'] + '_' + df_clean['Model']

# Optional: Filter the cleaned dataframe to only include makes with 50 or more occurrences
df_clean = df_clean[df_clean['Make'].isin(makes_over_50.index)]

# Display the first few rows to verify the results
print(df_clean.head())

         Event.Id Investigation.Type Accident.Number  Event.Date  \
0  20001218X45444           Accident      SEA87LA080  1948-10-24   
1  20001218X45447           Accident      LAX94LA336  1962-07-19   
2  20061025X01555           Accident      NYC07LA005  1974-08-30   
3  20001218X45448           Accident      LAX96LA321  1977-06-19   
4  20041105X01764           Accident      CHI79FA064  1979-08-02   

          Location        Country   Latitude   Longitude Airport.Code  \
0  MOOSE CREEK, ID  United States        NaN         NaN          NaN   
1   BRIDGEPORT, CA  United States        NaN         NaN          NaN   
2    Saltville, VA  United States  36.922223  -81.878056          NaN   
3       EUREKA, CA  United States        NaN         NaN          NaN   
4       Canton, OH  United States        NaN         NaN          NaN   

  Airport.Name  ... Total.Fatal.Injuries Total.Serious.Injuries  \
0          NaN  ...                  2.0                    0.0   
1          NaN  ..

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [47]:
import pandas as pd

# Load the data
df = pd.read_csv('data/AviationData.csv', low_memory=False)

# Replace 'UNK', 'NONE', and NaN values with 'Unknown' in 'Engine.Type'
df['Engine.Type'] = df['Engine.Type'].replace(['UNK', 'NONE', pd.NA], 'Unknown')
df['Engine.Type'] = df['Engine.Type'].fillna('Unknown')

# Columns to inspect
columns_to_inspect = [
    'Engine.Type',
    'Weather.Condition',
    'Number.of.Engines',
    'Purpose.of.flight',
    'Broad.phase.of.flight'
]

# Inspect each column individually
for col in columns_to_inspect:
    print(f"\n{'='*40}\nInspecting column: {col}\n{'='*40}")
    
    # Display basic statistics
    print("Basic Info:")
    print(df[col].describe(include='all'))
    
    # Display unique values and their counts
    print("\nValue Counts:")
    print(df[col].value_counts(dropna=False).head(20))  # Showing top 20 most frequent values
    
    # Check number of missing values
    missing_values = df[col].isnull().sum()
    print(f"\nMissing Values: {missing_values}")


Inspecting column: Engine.Type
Basic Info:
count             88889
unique               10
top       Reciprocating
freq              69530
Name: Engine.Type, dtype: object

Value Counts:
Engine.Type
Reciprocating      69530
Unknown             9150
Turbo Shaft         3609
Turbo Prop          3391
Turbo Fan           2481
Turbo Jet            703
Geared Turbofan       12
Electric              10
LR                     2
Hybrid Rocket          1
Name: count, dtype: int64

Missing Values: 0

Inspecting column: Weather.Condition
Basic Info:
count     84397
unique        4
top         VMC
freq      77303
Name: Weather.Condition, dtype: object

Value Counts:
Weather.Condition
VMC    77303
IMC     5976
NaN     4492
UNK      856
Unk      262
Name: count, dtype: int64

Missing Values: 4492

Inspecting column: Number.of.Engines
Basic Info:
count    82805.000000
mean         1.146585
std          0.446510
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
ma

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [50]:
import pandas as pd

# Load the data
df = pd.read_csv('data/AviationData.csv', low_memory=False)

# Display the number of non-null entries per column clearly
non_null_counts = df.count()
for column, count in non_null_counts.items():
    print(f"{column}: {count}")

# Remove columns with fewer than 20,000 non-null entries
threshold = 20000
df_cleaned = df.loc[:, df.count() >= threshold]

# Display the columns remaining after removal
print("\nColumns remaining after removal:")
print(df_cleaned.columns.tolist())

Event.Id: 88889
Investigation.Type: 88889
Accident.Number: 88889
Event.Date: 88889
Location: 88837
Country: 88663
Latitude: 34382
Longitude: 34373
Airport.Code: 50132
Airport.Name: 52704
Injury.Severity: 87889
Aircraft.damage: 85695
Aircraft.Category: 32287
Registration.Number: 87507
Make: 88826
Model: 88797
Amateur.Built: 88787
Number.of.Engines: 82805
Engine.Type: 81793
FAR.Description: 32023
Schedule: 12582
Purpose.of.flight: 82697
Air.carrier: 16648
Total.Fatal.Injuries: 77488
Total.Serious.Injuries: 76379
Total.Minor.Injuries: 76956
Total.Uninjured: 82977
Weather.Condition: 84397
Broad.phase.of.flight: 61724
Report.Status: 82505
Publication.Date: 75118

Columns remaining after removal:
['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date', 'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 'Injury.Severity', 'Aircraft.damage', 'Aircraft.Category', 'Registration.Number', 'Make', 'Model', 'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [56]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df = pd.read_csv('data/AviationData.csv', low_memory=False)

# Calculate the estimated number of total passengers
df['total_passengers'] = (df['Total.Fatal.Injuries'] + 
                          df['Total.Serious.Injuries'] + 
                          df['Total.Minor.Injuries'] + 
                          df['Total.Uninjured'])

# Avoid division by zero by filtering out rows with zero total passengers
df = df[df['total_passengers'] > 0]

# Create columns for percent fatal and serious injuries
df['percent_fatal'] = df['Total.Fatal.Injuries'] / df['total_passengers']
df['percent_serious'] = df['Total.Serious.Injuries'] / df['total_passengers']

# Create column for tracking if the aircraft was destroyed or not
df['destroyed'] = df['Aircraft.damage'].fillna('Unknown') == 'Destroyed'

# Clean up the 'Make' column by capitalizing the first letter of each word
df['Make'] = df['Make'].str.title()

# Get counts of each unique make
make_counts = df['Make'].value_counts()

# Filter makes with 50 or more occurrences
makes_over_50 = make_counts[make_counts >= 50]

# Replace 'UNK', 'NONE', and NaN values with 'Unknown' in 'Engine.Type'
df['Engine.Type'] = df['Engine.Type'].replace(['UNK', 'NONE', pd.NA, np.nan], 'Unknown')

# Remove rows with NaN values in the 'Model' column
df_cleaned = df.dropna(subset=['Model']).copy()

# Determine if each Model is unique to exactly one Make
# Count how many unique Makes are associated with each Model
model_make_counts = df_cleaned.groupby('Model')['Make'].nunique()

# Create a boolean column indicating if the model is unique to one Make
df_cleaned['Model_Unique_To_Make'] = df_cleaned['Model'].map(model_make_counts) == 1

# Create a derived column with a unique identifier combining Make and Model
df_cleaned['Make_Model_ID'] = df_cleaned['Make'] + '_' + df_cleaned['Model']

# Find and Remove columns with fewer than 20,000 non-null entries
non_null_counts = df.count()
threshold = 20000
df_cleaned = df_cleaned.loc[:, df_cleaned.count() >= threshold]

# Calculate the date 40 years ago from the current date
forty_years_ago = datetime.now() - timedelta(days=40*365)

# Create a filtered copy of the dataframe
df_filtered = df_cleaned.copy()

# Convert Event.Date column to datetime format
df_filtered['Event.Date'] = pd.to_datetime(df_filtered['Event.Date'])

# Filter only category Airplane, Amateur Built is No, and Event Date not more than 40 years ago
df_filtered = df_filtered[(df_filtered['Aircraft.Category'] == 'Airplane') & 
                          (df_filtered['Amateur.Built'] == 'No') & 
                          (df_filtered['Event.Date'] >= forty_years_ago)]

# Save the filtered data to a new CSV file
df_filtered.to_csv('data/AviationDataClean.csv', index=False)